In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window

### **Data Reading**

In [0]:
df = spark.read.format("parquet")\
    .load("abfss://bronze@fmcdatabrickse2e.dfs.core.windows.net/orders")

In [0]:
df.printSchema()

In [0]:
df = df.drop("_rescued_data")
display(df)

In [0]:
df = df.withColumn("order_date", to_timestamp(col("order_date")))
df.display()

In [0]:
df = df.withColumn("year", year(col("order_date")))
df.display()

In [0]:
df1 = df.withColumn("flag", dense_rank().over(Window.partitionBy("year").orderBy(col("total_amount").desc())))
df1.display()


In [0]:
df1 = df1.withColumn("rank_flag", rank().over(Window.partitionBy("year").orderBy(col("total_amount").desc())))
df1.display()

In [0]:
df1 = df1.withColumn("row_flag", row_number().over(Window.partitionBy("year").orderBy(col("total_amount").desc())))
df1.display()

### **OOP**

In [0]:
class Windows:

  def dense_rank(self, df):
    return df.withColumn("flag", dense_rank().over(Window.partitionBy("year").orderBy(col("total_amount").desc())))
  
  def rank(self, df):
    return df.withColumn("rank_flag", rank().over(Window.partitionBy("year").orderBy(col("total_amount").desc())))
  
  def row_number(self, df):
    return df.withColumn("row_flag", row_number().over(Window.partitionBy("year").orderBy(col("total_amount").desc())))

In [0]:
df_new = df
df_new.display()

In [0]:
obj = Windows()
df_new = obj.dense_rank(df_new)

### **Data Writing**

In [0]:
df.write.format("delta").mode("overwrite").save("abfss://silver@fmcdatabrickse2e.dfs.core.windows.net/orders")

In [0]:
%sql
CREATE TABLE IF NOT EXISTS databricks_cat.silver.orders_silver
USING DELTA
LOCATION 'abfss://silver@fmcdatabrickse2e.dfs.core.windows.net/orders'